In [21]:
from course_helpers import mlflow_connect

In [22]:
my_experiment = mlflow_connect()

OK - mlflow server is up and running. Setting Tracking URI to http://localhost:5000. Setting Experiment to 'Spam Detection'


In [23]:
import mlflow

mlflow_client = mlflow.tracking.MlflowClient()
runs = mlflow_client.search_runs(
    experiment_ids=my_experiment.experiment_id,
    filter_string="tags.mlflow.runName = 'Basic Spam Classification Model'",
)
if len(runs) == 0:
    raise Exception(
        f"No runs found for run name 'Basic Spam Classification Model' in experiment id {my_experiment.experiment_id}"
    )
my_run = runs[0]
print(f"Found run {my_run.info.run_id} with name '{my_run.info.run_name}'")

Found run 1f4a98ed892a48abb2824e6ea2dc104c with name 'Basic Spam Classification Model'


In [27]:
content_df = pd.DataFrame(
    {
        "content": [
            "Click this link and subscribe to my channel!",
            "This is the best ever video about MLflow!",
        ]
    }
)

artifact_path = f"runs:/{my_run.info.run_id}/spam_pipeline_model"
print(f"Loading model from {artifact_path}")
print()

reloaded_model = mlflow.pyfunc.load_model(artifact_path)
predictions = reloaded_model.predict(content_df)

pred_df = content_df.assign(prediction=predictions)
pred_df.head()

Loading model from runs:/1f4a98ed892a48abb2824e6ea2dc104c/spam_pipeline_model



,content,prediction
0,Click this link and subscribe to my channel!,1
1,This is the best ever video about MLflow!,0


In [28]:
import time

MODEL_NAME_IN_REGISTRY = "Spam Classifier"

try:
    mlflow_client.delete_registered_model(name=MODEL_NAME_IN_REGISTRY)
except Exception:
    pass

mlflow.register_model(model_uri=artifact_path, name=MODEL_NAME_IN_REGISTRY)
time.sleep(2)

model_info = mlflow_client.get_model_version(name=MODEL_NAME_IN_REGISTRY, version=1)
print(
    f"{model_info.name} version {model_info.version} is in stage '{model_info.current_stage}'"
)

Successfully registered model 'Spam Classifier'.
2023/10/26 07:12:21 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation. Model name: Spam Classifier, version 1
Created version '1' of model 'Spam Classifier'.


Spam Classifier version 1 is in stage 'None'


In [29]:
updated_model_info = mlflow_client.transition_model_version_stage(
    name=MODEL_NAME_IN_REGISTRY, version=1, stage="Production"
)
print(
    f"{updated_model_info.name} version {updated_model_info.version} is now in stage '{updated_model_info.current_stage}'"
)

Spam Classifier version 1 is now in stage 'Production'


In [35]:
for model_version in mlflow_client.search_model_versions(
    f"name='{MODEL_NAME_IN_REGISTRY}'"
):
    print(
        f"Model '{model_version.name}' version {model_version.version} is in stage '{model_version.current_stage}'. Artifact path: {model_version.source}"
    )

Model 'Spam Classifier' version 1 is in stage 'Production'. Artifact path: /workspaces/mlops-course/.mlflow_data/artifacts/1/1f4a98ed892a48abb2824e6ea2dc104c/artifacts/spam_pipeline_model
